# Pràctiques de Nous Usos de la Informàtica

**Nom de les persones del grup:** Xavi Cano & Orlando Manjarrez

# Pràctica 2. Recomanadors

# Construcció d'un recomanador.

In [97]:
# lectura de dades

import pandas as pd
import numpy as np

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

data = pd.merge(pd.merge(ratings, users), movies)
data = data[data.user_id < 100]
data = data[data.movie_id < 100]

**Alerta**: Les implementacions dels exercicis 6, 7 i 8 poden tardar molt en executar-se, considera fer-ho en un subset de les dades originals. En la 1a cel·la:
```
    data = data[data.user_id < 100]
    data = data[data.movie_id < 100]
```
El codi anterior limitaria les dades a 100 usuaris i 100 películes. Recorda re-executar les cel·les.

Com a guia, una implementació que usi N usuaris i películes, per l'exercici 6, ot arribar a trigar:

* N=100, 5 segons a 60 segons
* N=1000, 15 minuts a 1 hora
* N=10000, 20 hores a 60 hores

segons la implementació utilitzada

El següent codi, donat el conjunt de dades, construeix un conjunt d'entrenament i un conjunt  de test:

In [115]:
# generem subconjunts de training i test
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.ix[sampled_ids, 'for_testing'] = True
    return df

data['for_testing'] = False
grouped = data.groupby('user_id', group_keys=False).apply(assign_to_set)
movielens_train = data[grouped.for_testing == False]
movielens_test = data[grouped.for_testing == True]
print movielens_train.shape
print movielens_test.shape

(319, 11)
(135, 11)


La següent funció `evaluate(estimate)`, donat un conjunt de dades d'entrenament i un conjunt de dades de test ens avalua la precisió d'un sistema de recomanació que li passem per paràmetre. Per a cadascun dels elements del conjunt de test haurem de pronosticar el seu valor i comparar-lo amb el valor real que l'usuari li ha asignat. La mesura que utilizarem per avaluar el sistema és la root-mean-square error (rsme)

In [99]:
# definim una funció per avaluar el resultat de la recomanació.

def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate,test=movielens_test):
    ids_to_estimate = zip(test['user_id'], test['movie_id'])
    estimated = np.array([estimate(u,i) for (u,i) in ids_to_estimate])
    real = test.rating.values
    nans = np.isnan(estimated)
    return compute_rmse(estimated[~nans], real[~nans])

### EXERCICI 4

+ Construeix dues funcions, `dist_euclid(x,y)` i `coef_pearson(x,y)`, que implementin la distància Euclidiana i el coeficient de correlació de Pearson entre dos vectors usant funcions de pandas. 

+ Escriu les funcions que calculin la semblança entre dos series d'un DataFrame de Pandas. S'utiltizaran per calcular les similituds entre usuaris o entre items:

    + ``def sim_euclid (data_frame, row1, row2)``
    Calcula els vectors representatius de cada fila, C1 i C2, amb les puntuacions de les columnes que estan presents en ambdós files. En el cas dels usuaris (files), això implica trobar les películes (columnes) que han puntuat tots dos.<br />Si no hi ha puntuacions en comú, retornar 0. En cas contrari, retornar ``1/(1+dist_euclid(C1, C2))``

    + ``def sim_pearson (data_frame, row1, row2)``
    Calcular els vectors representatius de cada fila, C1 i C2, amb les puntuacions de les columnes que estan presents en ambdós files.<br />Si no hi ha puntuacions en comú, retornar 0. Retornar ``coef_pearson(C1,C2)``
    

In [106]:
import math
import numpy as np
import pandas as pd

# Returns the euclidean distance of two vectors
def dist_euclid(x, y):
    if len(x) == len(y):
        resultado = math.sqrt(sum([(x[i]-y[i])**2 for i in range(0,len(x))]))
    else:
        return 0
    return resultado

# Returns the Pearson correlation of two vectors 
def coef_pearson(x, y):
    if len(x) == len(y):
        denominador = math.sqrt(sum([(x[i]-np.mean(x))**2 for i in range(0,len(x))])) * math.sqrt(sum([(y[i]-np.mean(y))**2 for i in range(0,len(y))]))
        # Si el denominador es 0 nos evitamos calcular l numerador
        if denominador == 0:
            return 0
        else :
            numerador = sum([(x[i]-np.mean(x))*(y[i]-np.mean(y)) for i in range(0,len(x))])
            resultado = numerador/denominador 
    else:
        return 0
    return abs(resultado)

# Returns a distance-based similarity score for person1 and person2 based on euclidean distance
def sim_euclid(data_frame, row1, row2):
    #Obtenim el dataFrames amb les pelicules i votació de cada usuari
    data_usr1 = data_frame[ data_frame['user_id'] == row1][['movie_id','rating']]
    data_usr2 = data_frame[ data_frame['user_id'] == row2][['movie_id','rating']]
    #Finalment obtenim únicament les votacions de les pel·lícules evaluades per els dos usuaris
    ints = pd.merge(data_usr1, data_usr2, how='inner',on='movie_id', suffixes=('_usr1', '_usr2')) 
    return 1/(1+dist_euclid(ints['rating_usr1'], ints['rating_usr2']))

# Returns a distance-based similarity score for person1 and person2 based on pearson distance
def sim_pearson(data_frame, row1, row2):
    #Obtenim el dataFrames amb les pelicules i votació de cada usuari
    data_usr1 = data_frame[ data_frame['user_id'] == row1][['movie_id','rating']]
    data_usr2 = data_frame[ data_frame['user_id'] == row2][['movie_id','rating']]
    #Finalment obtenim únicament les votacions de les pel·lícules evaluades per els dos usuaris
    ints = pd.merge(data_usr1, data_usr2, how='inner',on='movie_id', suffixes=('_usr1', '_usr2'))
    return coef_pearson(ints['rating_usr1'], ints['rating_usr2'])

Tests de les funcions, pots realitzar modificacions prèvies a les taules (per exemple, agrupar o pivotar) per accelerar el procés

In [108]:
# Ejemplo con dos vectores para comprobar el buen funcionamiento 
# de las funciones dist_euclid y el coef_pearson
print dist_euclid ([2,2,2,2],[1,1,1,1])
print coef_pearson ([2,9,8,2],[1,7,7,3])

# Execute functions
print sim_euclid(data, 1, 2)
print sim_pearson(data, 1, 2)

2.0
0.956606702325
1.0
0


### EXERCICI 5

+ Feu dues funcions, ``get_best_euclid(data_frame, user, n)`` i ``get_best_pearson(data_frame, user, n)``, que retornin els ``n`` usuaris més semblants segons aquestes dues mesures de similitud.

In [111]:
# return the N most similar users to a given user based on euclidean distance
def get_best_euclid(data_frame, user, n):
    # Per tal de fer-ho més eficient reduim el DataFrame a unicament les pel·licules votades per 'user'
    data2 = pd.merge(data_frame,data_frame[data_frame['user_id']==user][['movie_id','title']],how='inner',on='movie_id')[['user_id','movie_id','rating']]
    # Obtenim una llista dels usuaris excepte el propi a comparar
    users = set(data2[data2['user_id'] != user]['user_id'])
    # Calculem la dist. Euclid. entre usuaris (i,user) per tots els users necessaris i els ordenem.
    rank = [(sim_euclid(data2,user,i),i) for i in users]   
    rank = sorted(rank, key=lambda pearson_rtg: pearson_rtg[0], reverse=True)
    return [ rank[i] for i in range(n) ]
    
# return the N most similar users to a given user based on pearson correlation
def get_best_pearson(data_frame, user, n):
    # Per tal de fer-ho més eficient reduim el DataFrame a unicament les pel·licules votades per 'user'
    data2 = pd.merge(data_frame,data_frame[data_frame['user_id']==user][['movie_id','title']],how='inner',on='movie_id')[['user_id','movie_id','rating']]
    # Obtenim una llista dels usuaris excepte el propi a comparar
    users = set(data2[data2['user_id'] != user]['user_id'])
    # Calculem el coef entre usuaris (i,user) per tots els users.
    rank = [(sim_pearson(data2,user,i),i) for i in users]
    rank = sorted(rank, key=lambda pearson_rtg: pearson_rtg[0], reverse=True)
    return [ rank[i] for i in range(n) ] 

In [112]:
# Execute functions
# Aquestes funcions poden trigar a executar-se; feu proves primer amb una part petita de la base de dades.

print get_best_euclid(data, 1, 5)
print get_best_pearson(data, 1, 5)

[(1.0, 9), (1.0, 19), (1.0, 34), (1.0, 36), (1.0, 44)]
[(0, 6), (0, 8), (0, 9), (0, 10), (0, 18)]


### EXERCICI 6

En l'exercici 6 i 7 es desenvoluparà un sistema de recomanació basat en usuaris i en ítems, respectivament.

El codi donat, que es basa en 3 classes, és la recomenada per fer-ho òptim i reaprofitar el màxim de codi, però s'acceptaran solucions que no la segueixin, sempre hi quan respectin el mètode "estimate" explicat més abaix i funcionin de forma correcte.

#### `CollaborativeFiltering`

Una classe base, comuna en els 2 recomanadors, que implementarà:
  
  * `__init__`: Rep com a paràmetres el dataframe (que constarà de `user_id`, `movie_id` i `rating`), la funció de semblança (Euclidiana o Pearson) que volem usar i un paràmetre `M` que indica el tamany que tindrà la matriu de similituds.
  
  * `precompute`: Generar per cada estimació la semblança entre 2 usuaris o items seria molt costós i faria l'algorisme molt lent, per tant, aquesta funció omplirà la taula MxM (on M es el número de usuaris o items, segons el recomanador) amb el coeficient de semblança.
      * Nota: La taula es un DataFrame de Pandas, per tant accedirem als element fent servir l'indexat de Pandas (que correspon al id del user/movie, i no a la posició 0...i)
  
  * `estimate`: s'encarrega de donar la predicció, en aquest cas donat un usuari i una pel·lícula estimar el seu rating.
    + Nota 1: Si un `user_id` o `movie_id` no es troba en el DataFrame, cal retornar "np.NAN"
    + Nota 2: En el recomenador d'usuaris, s'ha d'evitar comparar `user_id` a ell mateix. De la mateixa forma, en el d'items evitarem comparar un `movie_id` amb sí mateix.

#### `UserRecomender`

Recomanador basat en usuaris que hereta de `CollaborativeFiltering`. Implementarà:

  * `__init__`: Pot realitzar transformacions al DataFrame
  
#### `ItemRecomender`

Recomanador basat en items que hereta de `CollaborativeFiltering`. Implementarà:

  * `__init__`: Pot realitzar transformacions al DataFrame
    

In [304]:
import os.path

class CollaborativeFiltering(object):
    """ Collaborative filtering using a custom sim(u,u'). """
    
    def __init__(self, data, M, similarity=sim_pearson):
        """ Constructor """
        self.sim_method = similarity # Gets recommendations for a person by using a weighted average
        self.df = data 
        self.sim = pd.DataFrame(0.0, index=M, columns=M)
        # Diccionario de las peliculas vistas por cada usuario para agilizar los calculos de busqueda 
        self.dicUser={} # key usuario y contiene sus peliculs y ratings de estas
        users=self.df.user_id.unique() # Obtener todos los ids de los usuarios del data
        for us in users: # Se guardan los valores de rating i movie_id de cada usuario para no buscar mas al DataFrame
            self.dicUser[us]=self.df.ix[self.df['user_id']==us,['rating','movie_id']].sort(ascending=False,columns='rating') #Ordre Rating
        # Diccionario de los usuarios y ratings de cada pelicula para agilizar los calculos de busqueda
        self.dicPelis={}#key pelicula y contiene sus ratings de cada usuario
        pelis=self.df.movie_id.unique() #Obtener todos los ids de la peliculas del data
        for film in pelis: #Se guardan los valores de Rating y user_id de cada pelicula para no buscar mas en el DataFrame
            self.dicPelis[film]=self.df.ix[self.df['movie_id']==film,['rating','user_id']].sort(ascending=False,columns='rating') #Ordre Rating

    def precompute(self, num):
        """Prepare data structures for estimation. Compute similarity matrix self.sim"""
        #Iterar el dataframe con iterrrows()
        diag = 1 # indice que ara la diagonal de la tabla ya que la matriz es simetrica
        # Guardamos en col todos los indices. Como la matriz es simetrica tan solo recorremos 
        # las posiciones superiores de la diagonal ya que las posiciones inferiores tendrn el mismo
        # valor, tan solo hemos de invertir x e y.
        for x, col in self.sim.iterrows(): 
            for y in col.index.values[diag:]: 
                var=self.sim_method(self.df,x,y) #hacer el calculo de person
                self.sim.set_value(x,y,var) #Guardar el valor en la tabla de similitudes
                self.sim.set_value(y,x,var) #Guardar el valor invirtiendo las coordenadas
            diag += 1
        
        if num == 1:
            #self.sim.save("MatrixUser") #Guardar la taula de similituts a disc
            print 1
        else:
            print 2
            #self.sim.save("ItemBased") #Guardar la tabla a disco para su posterior recuperacion
            
    def estimate(self, row, col, num):
        """ Given an row (user_id in 6, movie_id in 7) and a column (movie_id in 6, user_id in 7) 
            returns the estimated rating """
        if num == 1:
            # Si el usuario no existe retornamos np.NAN
            if row not in self.dicUser or col not in self.dicPelis:
                return np.NAN,' El usuario no existe'
            #Al tener que cargar varias veces por id de movie se usa el diccionario de peliculas para agilizar la carga
            pearsonUser = [self.sim.loc[row,user]  for user in self.dicPelis[col]['user_id']]
            ratingMovie = [rating for rating in self.dicPelis[col]['rating']]
            #estimate rating
            sumPearsonXRating = sum(np.multiply(pearsonUser,ratingMovie))
            sumPears = sum(pearsonUser)
            #Filtro para evitar valores nan y los divisores con valor absoluto entre 0 i 0,5 ya que generan errores
            if(sumPearsonXRating == 0 or abs(sumPears) < 0.5):
                return 3;
            out=int(round(sumPearsonXRating/sumPears))
            if out> 5: #Filtro para no sacar un valor superior a 5 ya que por el redondeo puede salir valor mas alto
                return 5
            else:
                return out
        else:
            # Si la pelicula no existe retornamos np.NAN
            if row not in self.dicUser or col not in self.dicPelis:
                return np.NAN
            #Al tener que hacer varias consultas sobre el mismo usuario se usa un diccionario para no perder mucho tiempo buscando en el dataFrame
            pearsonMovie = [self.sim.loc[col,movie]  for movie in self.dicUser[row]['movie_id']]
            ratingMovie = [rating for rating in self.dicUser[row]['rating']]#Covertir en una lista todas las puntuacions 
            #estimate rating
            sumPearsonXRating = sum(np.multiply(pearsonMovie,ratingMovie))
            sumPears = sum(pearsonMovie)
            #Se filtra el valor absolut de sumPears ya que si el valor esta entre 0 y 0.5 genera error
            if(sumPearsonXRating == 0 or abs(sumPears) < 0.5):
                return 3;
            out=int(round(sumPearsonXRating/sumPears))
            if out > 5: #Filtro por si el valor tiene numeros decimales que lo sacan de la escala
                return 5;
            else:
                return out
            

In [315]:
class UserRecomender(CollaborativeFiltering):
    """ Recomender using Collaborative filtering with a User similarity (u,u'). """
    
    def __init__(self, data_train, similarity=sim_pearson):
        """ Constructor """
        
        # You should do any transformation to data_train (grouping/pivot/...) here, if needed
        transformed_data = data_train
        
        super(UserRecomender, self).__init__(transformed_data, data_train.user_id.unique(), similarity)

                
    def estimate(self, user_id, movie_id):
        """ Given an user_id and a movie_id returns the estimated rating for such movie """
        return super(UserRecomender, self).estimate(user_id, movie_id, 1)
        
    #Exercici 8
    def get_recomendations(self, user_id, n): #Nou metode per el Ex 8
        out=[]
        x=0
        vistes=self.dicUser[user_id] #Peliculas ya vistas por el usuario
        #Se eliminan usuarios con valor negativo o proximo al 0 y se ordenan para asi ir obteniendo ha que usuarios consultar primero
        ids=list(self.sim.ix[self.sim[user_id]>0.50,user_id].order(ascending=False).index.values) #llista dels millors candidats
        numus=len(ids) #numero de candidatos
        while(x<numus):
            #Filtrage de peliculas con puntuacion de 5 de un usuario, però en caso que ningun usuario puntue con 5 seria intresante bajar este tope
            millors=list(self.df.ix[(self.df['user_id']==ids[x]) & (self.df['rating']>4)].sort(ascending=False,columns='rating')['movie_id'])
            iguals=set(millors) & set(vistes) #Se compruevan si estan repetidas las peliculas para llenar las lista con valores unicos
            for elim in iguals:
                millors.remove(elim)#Se eliminan las peliculas repetidas
            out.extend(millors) #añadir las peliculas a la lista de salida
            out=list(set(out))#Eliminar posibles repeticiones
            if(len(out)>n):
                return out[:n] #Retorna la lista de n peliculas recomendadas
            x=x+1 #siguiente usuario
        
        return out #En caso de no tener mas referencia devuelve todo lo guardado

In [316]:
user_reco = UserRecomender(movielens_train)
user_reco.precompute(1)
user_reco.estimate(user_id=2, movie_id=1)

C:\Users\orla_\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\orla_\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


1


3

In [307]:
evaluate(user_reco.estimate, movielens_test)

ValueError: cannot set an array element with a sequence

### EXERCICI 7

+ Desenvolupa un sistema de recomanació col·laboratiu basat en ítems. Si la classe `CollaborativeFiltering` s'ha fet prou genèrica, tan sols caldrà fer petites modificacions a `__init__`, del contrari, podeu fer les modificacions que cregueu necessàries.

In [308]:
class ItemRecomender(CollaborativeFiltering):
    """ Recomender using Collaborative filtering with a Item similarity (i,i'). """
    
    def __init__(self,data_train, similarity=sim_pearson):
        """ Constructor """
        
        # You should do any transformation to data_train (grouping/pivot/...) here, if needed
        transformed_data = data_train
        
        super(ItemRecomender, self).__init__(transformed_data, data_train.movie_id.unique(), similarity)

            
    def estimate(self, user_id, movie_id):
        """ Given an user_id and a movie_id returns the estimated rating for such movie """
        return super(ItemRecomender, self).estimate(movie_id, user_id, 0)
    
    def get_recomendations(self, user_id, n): 
        out=[]
        i,x=0,0
        vistas=list(self.dicUser[user_id].movie_id.unique()) #Array de peliculas ya vistas por el usuario
        #Comentar que se ha puesto que busque las notas superiores al tres porque puede que no haya puntuado un 5
        Pelis=np.array(data.ix[(data['user_id']==user_id) & (data['rating']>3)].sort(ascending=False,columns='rating')['movie_id'])
        numPelis=len(Pelis)
        #Se iteran las peliculas con referencia a las de la lista de similitudes
        while(x<numPelis):
            #Extraer las peliculas con un valor de similitud muy alto
            #Si no se obtiene peliculas se tendria que bajar el nivel de similitud para mostrar peliculas
            Reco=list(self.sim.ix[self.sim[Pelis[x]]>0.50,Pelis[x]].order(ascending=False).index.values)
            iguals=set(Reco) & set(vistas) #Eliminar las peliculas iguales en las dos listas
            for elim in iguals:
                Reco.remove(elim) #Eliminar las peliculas iguales para que las que hayan en out sean unicas y no vistas
            out.extend(Reco)#Se añaden a la lista de salia quitando los ids repetidos
            out=list(set(out))#Se quitan los valores repetidos
            if(len(out)>n):
                return out[:n] #Devuelve los n valores
            x=x+1 #Siguiente pelicula
        return out  
    

In [309]:
item_reco = ItemRecomender(movielens_train)
item_reco.precompute(0)
item_reco.estimate(user_id=2, movie_id=1)

C:\Users\orla_\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\orla_\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


2


3

In [310]:
evaluate(item_reco.estimate, movielens_test)

1.4591203713417602

### EXERCICI 8

* Feu un nou mètode `get_recomendations(user_id, n)` que retorni les n pel·lícules recomenades per a l'usuari user_id. De nou, és recomenable fer-ho a la clase pare, `CollaborativeFiltering`, cridant-la des dels fills de forma semblant a com fa `estimate`.

* Executeu la funció en els dos recomenadors 

In [319]:
user_reco.get_recomendations(8, 5)

C:\Users\orla_\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: FutureWarning: order is deprecated, use sort_values(...)
C:\Users\orla_\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


[1, 34, 36, 17, 50]

In [312]:
item_reco.get_recomendations(1, 5)

C:\Users\orla_\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\orla_\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: FutureWarning: order is deprecated, use sort_values(...)


[34, 35, 36, 6, 73]